In [1]:
import numpy as np
import os
import matplotlib
from matplotlib.font_manager import fontManager
#for i in sorted(fontManager.get_font_names()):
#    print(i)
matplotlib.rc('font', family='Microsoft JhengHei')
import cv2
from tensorflow.keras.models import load_model

In [2]:
def dark_channel(image) : 
    H, W, _ = image.shape
    patch_size = 15
    pad_size = patch_size // 2
    #創建 H*W 全零矩陣
    dc = np.zeros((H, W), dtype=np.float32)
    #用無限大的值填充的用意是取local min才不會影響到取值
    imJ = np.pad(image ,((pad_size, pad_size), (pad_size, pad_size), (0, 0)), mode='constant', constant_values=np.inf)   
    #計算暗通道
    for j in range(H):
        for i in range(W):
            #遍例3通道, 所有patch為15
            patch = imJ[j:(j+patch_size), i:(i+patch_size),:]
            #將patch中抓到的local min存到dc(j, i)裡
            dc[j, i] = np.min(patch)
            
    return dc


def atmospheric_light(image, dark_channel):
	#在暗通道中找最亮的像素，並在原彩圖中找到對應的位置，來計算大氣光的顏色
	H, W, _ = image.shape
	imsize = H * W													#計算圖像總像素數量
	numpx = np.floor(imsize / 1000).astype(int)						#計算要選擇的像素數量，選前0.1%，並向下取整
	dark_channel_Vec = dark_channel.ravel()							#將dark_channel展平成 imsize *1 的列向量
	ImVec = image.reshape(imsize, 3)								#將原始影像image展平成 imsize * 3 的矩陣
	indices = np.argsort(dark_channel_Vec)							#對暗通道進行升序排序，indices=索引值
	indices = indices[-numpx:]										#選擇排序後最亮的0.1%，計算從哪裡開始提取索引的起點，並到end
	atmSum = np.zeros(3, dtype=np.float32)							#計算大氣光的顏色，創建 1 * 3 全0矩陣，用來儲存最亮像素的顏色值累加和

	#遍例所有選中的像素
	for ind in range(numpx):
		atmSum += ImVec[indices[ind]]								#將每個選中像素的RGB顏色值累加到atmSum

	A = atmSum / numpx												#將atmSum取平均，即為大氣光的顏色

	return A


def fusion(original, dehaze_06, dehaze_12, dehaze_15):
    original = original
    dehaze_06 = dehaze_06
    dehaze_12 = dehaze_12
    dehaze_15 = dehaze_15

    height, width, channels = original.shape
    I = np.zeros((height, width, channels, 4), dtype=np.float32)
    I[:, :, :, 0] = original
    I[:, :, :, 1] = dehaze_06
    I[:, :, :, 2] = dehaze_12
    I[:, :, :, 3] = dehaze_15

    r, c, _, N = I.shape

    # 將對比度和飽和度結合為權重圖
    W = np.ones((r, c, 3, N), dtype=np.float32) * contrast(I) * saturation(I)

    # 加個 eps 避免除0 & 歸一化 
    W = W + 1e-12
    W = W / np.tile(np.sum(W, axis=3)[:, :, :, np.newaxis], (1, 1, 1, N))   #或是這樣也可以 W = W / (np.sum(W, axis=2, keepdims=True))

    # 創建一個空金字塔
    pyr = gaussian_pyramid(np.zeros((r, c, 3), dtype=np.float32))
    nlev = len(pyr)

    # 多分辨率融合
    for i in range(N):
        # 從每個輸入圖像構建金字塔
        pyrW = gaussian_pyramid(W[:, :, :, i])
        pyrI = laplacian_pyramid(I[:, :, :, i])

        # 融合
        for l in range(nlev):
            pyr[l] = pyr[l] + pyrW[l] * pyrI[l]

    # 重建
    R = reconstruct_laplacian_pyramid(pyr)
    R_clip = np.clip(R, 0, 1)
    
    return R_clip


def contrast(I):
    h = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]], dtype=np.float32)  # Laplacian filter
    N = I.shape[3]
    C = np.zeros((I.shape[0], I.shape[1], 3, N), dtype=np.float32)

    for i in range(N):
        mono = cv2.cvtColor(I[:, :, :, i], cv2.COLOR_BGR2GRAY)
        mono = cv2.cvtColor(mono, cv2.COLOR_GRAY2BGR)
        C[:, :, :, i] = np.abs(cv2.filter2D(mono, -1, h, borderType=cv2.BORDER_REPLICATE))

    return C


def saturation(I):
    N = I.shape[3]
    C = np.zeros((I.shape[0], I.shape[1], 3, N), dtype=np.float32)

    for i in range(N):
        # 飽和度計算為顏色通道的標準差
        R = I[:, :, 0, i]
        G = I[:, :, 1, i]
        B = I[:, :, 2, i]
        mu = (R + G + B) / 3
        sat = np.sqrt(((R - mu)**2 + (G - mu)**2 + (B - mu)**2) / 3)
        sat = cv2.cvtColor(sat, cv2.COLOR_GRAY2BGR)
        C[:, :, :, i] = sat

    return C


def pyramid_filter():
    return np.array([0.0625, 0.25, 0.375, 0.25, 0.0625])
filter = pyramid_filter()


def downsample(I, filter):
    # 與自定義 filter 進行卷積
    R = cv2.filter2D(I, -1, np.expand_dims(filter,axis=0), borderType=cv2.BORDER_REFLECT)     # 水平
    R = cv2.filter2D(R, -1, np.expand_dims(filter,axis=1), borderType=cv2.BORDER_REFLECT)     # 垂直

    # 下採樣
    R = R[::2, ::2, :]
    return R


def upsample(I, odd, filter):
    # 增加分辨率
    I = np.pad(I, [(1, 1), (1, 1), (0, 0)], mode='edge')  # 用1像素邊界填充圖像
    r, c, k = 2 * I.shape[0], 2 * I.shape[1], I.shape[2]
    R = np.zeros((r, c, k))

    R[0:r:2, 0:c:2, :] = 4 * I                          # R[0:r:2, 0:c:2, :] 和 R[::2, ::2, :] 是一樣意思的
                                                        # 4 * I 可以看作是一種插值方式，將原始像素的值擴展到更大的區域，
                                                        # 以維持亮度或顏色的一致性，這方法可以幫助在上採樣過程中減少亮度或顏色的損失。
    # 插值，與可分離濾波器進行卷積
    R = cv2.filter2D(R, -1, np.expand_dims(filter,axis=0), borderType=cv2.BORDER_REFLECT)   # 就算不指定填充邊界的方式，cv2.filter2D默認使用cv2.BORDER_CONSTANT
    R = cv2.filter2D(R, -1, np.expand_dims(filter,axis=1), borderType=cv2.BORDER_REFLECT)

    # 刪除邊界
    R = R[2:r-2-odd[0], 2:c-2-odd[1], :]
    return R


def gaussian_pyramid(I, nlev=None):
    r = I.shape[0]
    c = I.shape[1]

    if nlev is None:
        # 計算金字塔最高層數
        nlev = int(np.floor(np.log2(min(r, c))))

    # 創建包含 nlev 個元素的全None列表，第0層=原始影像
    pyr = [None] * nlev
    pyr[0] = I.copy()

    # 遞迴式的下採樣
    for l in range(1, nlev):
        pyr[l] = downsample(pyr[l-1], filter)

    return pyr


def laplacian_pyramid(I, nlev=None):
    r = I.shape[0]
    c = I.shape[1]

    if nlev is None:
        # 計算金字塔最高層數
        nlev = int(np.floor(np.log2(min(r, c))))

    # 遞迴建立金字塔
    pyr = [None] * nlev
    filter = pyramid_filter()
    J = I.copy()

    for l in range(nlev - 1):
        # 應用低通濾波器，然後下採樣
        I = downsample(J, filter)
        odd = (2 * I.shape[0] - J.shape[0], 2 * I.shape[1] - J.shape[1])  # 檢查上採樣版本是否需要奇數
        # 在每一層中，存儲圖像和上採樣低通版本之間的差異
        pyr[l] = J - upsample(I, odd, filter)

        J = I  # 繼續使用低通圖像

    pyr[nlev - 1] = J  # 最粗糙的層包含剩餘的低通圖像

    return pyr


def reconstruct_laplacian_pyramid(pyr):
    nlev = len(pyr)
    r, c, _ = pyr[0].shape

    R = pyr[nlev - 1].copy()
    filter = pyramid_filter()

    for l in range(nlev - 2, -1, -1):
        odd = (2 * R.shape[0] - pyr[l].shape[0], 2 * R.shape[1] - pyr[l].shape[1])
        R = pyr[l] + upsample(R, odd, filter)
    R = R.astype(np.float32)

    return R

In [3]:
def restore_transmission_dehaze(original_norm, d_GF, A, C, beta):

    t = np.exp(-beta*d_GF)
    selected_t = np.clip(t, 0.1, 1)

    I_dehaze = np.zeros_like(original_norm)
    for i in range(C):
        I_dehaze[:, :, i] = (original_norm[:, :, i] - A[0, 0, i]) / (selected_t) + A[0, 0, i]
    I_dehaze = np.clip(I_dehaze, 0, 1)

    return original_norm, d_GF, selected_t, I_dehaze

In [4]:
from ultralytics import YOLO
import random

YOLO_model =  YOLO("yolo11x.pt")

def use_YOLO(image):
    
    target_classes = {"car", "bus", "truck", "person", "motorcycle", "bicycle", "traffic light", "handbag", 
                    "orange", "cow", "skateboard", "dog", "bench", "umbrella", "surfboard", "frisbee", "banana", 
                    "backpack", "suitcase", "airplane", "train","bear", "horse", "clock", "fire hydrant", "bird"}
    class_colors = {
    'car': (0, 255, 0),             # 綠色
    'bus': (255, 0, 0),             # 紅色
    'truck': (0, 0, 255),           # 藍色
    'person': (255, 255, 0),        # 黃色
    'motorcycle': (255, 0, 255),    # 洋紅（紫紅）
    'bicycle': (0, 255, 255),       # 青色（湖藍）
    'traffic light': (255, 165, 0), # 橘色
    'handbag': (128, 0, 128),       # 紫色
    'orange': (255, 140, 0),        # 深橘（Dark Orange）
    'cow': (139, 69, 19),           # 棕色（Saddle Brown）
    'skateboard': (0, 128, 128),    # 藍綠色（Teal）
    'dog': (160, 82, 45),           # 黃褐色（Sienna）
    'bench': (105, 105, 105),       # 深灰色（Dim Gray）
    'umbrella': (0, 100, 0),        # 深綠色（Dark Green）
    'surfboard': (255, 105, 180),   # 粉紅色（Hot Pink）
    'frisbee': (75, 0, 130),        # 靛藍（Indigo）
    'banana': (255, 255, 102),      # 淺黃（Light Yellow）
    'backpack': (70, 130, 180),     # 鋼藍色（Steel Blue）
    'suitcase': (199, 21, 133),     # 中紫紅（Medium Violet Red）
    'airplane': (0, 191, 255),      # 深天藍
    'train': (255, 215, 0),         # 金黃色（Gold）
    'bear': (128, 128, 0),          # 橄欖色（Olive）
    'horse': (220, 20, 60),         # 猩紅色（Crimson）
    'clock': (0, 0, 139),           # 深藍色（Dark Blue）
    'fire hydrant': (255, 20, 147), # 深粉紅（Deep Pink）
    'bird': (34, 139, 34)           # 森林綠（Forest Green）
}

    def get_color(label):
        if label not in class_colors:
            class_colors[label] = tuple(random.randint(0, 255) for _ in range(3))
        return class_colors[label]
    
    results = YOLO_model(image)
    for result in results:
        boxes = result.boxes
        for box in boxes:
            cls_id = int(box.cls[0])
            conf = float(box.conf[0])
            label = YOLO_model.names[cls_id]
            
            if label.lower() in target_classes:
                color = get_color(label)
                print(f"偵測到{label} (confidence: {conf:.2f})")
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)#, (0, 255, 0), 2)
                # cv2.putText(image, f"{label} {conf:.2f}", (x1, y1 - 10),
                cv2.putText(image, f"{label}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)#, (0, 255, 0), 2)
            else:
                print(f"\033[91m忽略類別: {label}\033[0m")
    
    return image

In [5]:
input_folder = 'data'
output_folder = 'dehaze and YOLO result'
os.makedirs(output_folder, exist_ok=True)
CNN_model = load_model('model+weights.h5', compile=False)

img_count = 0

# 遍歷資料夾A中的所有 PNG 檔案
for filename in os.listdir(input_folder):
    if filename.endswith('.png'):
        image_name = os.path.splitext(filename)[0]
        image_path = os.path.join(input_folder, filename)
        # print(image_name, "\n")
        image_output_folder = os.path.join(output_folder, image_name)
        
        original = cv2.imread(f'{image_path}')
        ori_resize = cv2.resize(original, (224, 224)) / 255.0
        original_norm = original.astype(np.float32) / 255.0
        gray_image = cv2.cvtColor(original_norm, cv2.COLOR_BGR2GRAY)
        CNN_input = np.expand_dims(ori_resize, axis=0)
        CNN_input = np.array(CNN_input)
        print(f"\033[33mCNN is predicting the depth of image : {image_name}......\033[0m")
        predictions = CNN_model.predict(CNN_input)
        d = predictions[0]
        H, W, C = original.shape
        d = cv2.resize(d, (W, H))
        radius = round(np.minimum(H, W) / 50)
        d_GF = cv2.ximgproc.guidedFilter(guide=gray_image, src=d, radius=radius, eps=1e-5)

        dc = dark_channel(original_norm)
        A = atmospheric_light(original_norm, dc)
        if A.shape == (3,):
            A = A.reshape(1, 1, 3)

        original_norm, d_GF, selected_t_06, I_dehaze_06 = restore_transmission_dehaze(original_norm, d_GF, A, C, beta=0.6)
        _            , _   , selected_t_12, I_dehaze_12 = restore_transmission_dehaze(original_norm, d_GF, A, C, beta=1.2)
        _            , _   , selected_t_18, I_dehaze_15 = restore_transmission_dehaze(original_norm, d_GF, A, C, beta=1.5)

        print("\033[33mPyramid fusion......\033[0m")
        fusion_result = fusion(original_norm, I_dehaze_06, I_dehaze_12, I_dehaze_15)
        fusion_result_8bit_255 = ((np.clip(fusion_result, 0, 1))*255).astype(np.uint8)

        color_d_GF_INFERNO = cv2.applyColorMap((255 - (np.clip(d_GF, 0, 1)*255)).astype(np.uint8), cv2.COLORMAP_INFERNO)
        ori_to_save = (np.clip(original_norm, 0, 1) * 255).astype(np.uint8)

        print("有霧圖偵測到的物件 : ")
        ori_use_yolo = use_YOLO(ori_to_save)
        print("去霧圖偵測到的物件 : ")
        dehaze_use_yolo = use_YOLO(fusion_result_8bit_255)

        concat_image = cv2.hconcat([ori_use_yolo, dehaze_use_yolo])
        concat_path = os.path.join(output_folder, f'{image_name}_concat (dehaze and YOLO).png')
        cv2.imwrite(concat_path, concat_image)
        print("\033[32mDone\033[0m", "\n")

        img_count += 1

print(f"總共處理了 {img_count} 張影像")


CNN is predicting the depth of image : 03125946_64f412c2409657785......
1/1 [==============================] - 4s 4s/step
Pyramid fusion......
有霧圖偵測到的物件 : 

0: 480x640 1 person, 554.8ms
Speed: 1.7ms preprocess, 554.8ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)
偵測到person (confidence: 0.91)
去霧圖偵測到的物件 : 

0: 480x640 1 person, 490.5ms
Speed: 1.9ms preprocess, 490.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
偵測到person (confidence: 0.91)
Done 

CNN is predicting the depth of image : aerial......
1/1 [==============================] - 0s 14ms/step
Pyramid fusion......
有霧圖偵測到的物件 : 

0: 480x640 (no detections), 444.4ms
Speed: 2.0ms preprocess, 444.4ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)
去霧圖偵測到的物件 : 

0: 480x640 (no detections), 464.8ms
Speed: 1.5ms preprocess, 464.8ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
Done 

CNN is predicting the depth of image : bear1......
1/1 [==========================

In [6]:
import pyttsx3

engine = pyttsx3.init()
engine.setProperty('rate', 150)
engine.setProperty('volume', 1.0)
engine.say("程式碼已執行完畢，快點來啦")
engine.runAndWait()